In [3]:
from confluent_kafka import Producer, KafkaError
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
import json

admin_client = AdminClient({'bootstrap.servers': 'kafka:9092'})

In [9]:
# LIST TOPICS

md = admin_client.list_topics(timeout=10)
print(" {} topics:".format(len(md.topics)))
for t in iter(md.topics.values()):
    if t.error is not None:
        errstr = ": {}".format(t.error)
    else:
        errstr = ""
    print("  \"{}\" with {} partition(s){}".format(t, len(t.partitions), errstr))

 1 topics:
  "__consumer_offsets" with 50 partition(s)


In [8]:
fs = admin_client.delete_topics(["DEALER1_TRUCKS","DEALER2_USERS","DEALER1_CARS","DEALER2_VEHICLES"], operation_timeout=30)

# Wait for operation to finish.
for topic, f in fs.items():
    try:
        f.result()  # The result itself is None
        print("Topic {} deleted".format(topic))
    except Exception as e:
        print("Failed to delete topic {}: {}".format(topic, e))

Failed to delete topic DEALER1_TRUCKS: KafkaError{code=UNKNOWN_TOPIC_OR_PART,val=3,str="Broker: Unknown topic or partition"}
Failed to delete topic DEALER2_USERS: KafkaError{code=UNKNOWN_TOPIC_OR_PART,val=3,str="Broker: Unknown topic or partition"}
Failed to delete topic DEALER1_CARS: KafkaError{code=UNKNOWN_TOPIC_OR_PART,val=3,str="Broker: Unknown topic or partition"}
Failed to delete topic DEALER2_VEHICLES: KafkaError{code=UNKNOWN_TOPIC_OR_PART,val=3,str="Broker: Unknown topic or partition"}


In [6]:
from confluent_kafka import Consumer, KafkaError
import json

c = Consumer(
    {
        'bootstrap.servers': 'kafka:9092',
        'group.id': 'test',
        'auto.offset.reset': 'earliest'
    }
)

c.subscribe(["DEALER1_CARS","DEALER1_TRUCKS"])

waiting = False

try:
    while True:
        msg = c.poll(1.0)                                        # retrieve records
        if msg is None:
            if waiting:
                print(".",end =" ")
            else:
                print("Waiting",end =" ")
                waiting = True
            continue
        elif msg.error():
            print('error: {}'.format(msg.error()))
            waiting = False
        else:
            value = json.loads(msg.value())                      # get the value of the message (json.loads)
            topic = msg.topic()                               # get the key of the message
            print("\nConsumed record from {} with value {}"
                  .format(topic, value))
            waiting = False
except KeyboardInterrupt:
    pass
finally:
    c.close()  

Waiting . . . . . . . . . . . . . 
Consumed record from DEALER1_CARS with value {'user': 'Molly Davis', 'rid': '1', 'manufacturer': 'Fiat', 'model': 'Panda', 'plate': 'FJ7PUJJ', 'status': 'START', 'ts': '2022-01-14 13:51:24'}

Consumed record from DEALER1_CARS with value {'user': 'Laura Baker', 'rid': '2', 'manufacturer': 'Tesla', 'model': 'Model S', 'plate': 'JFGJ60A', 'status': 'START', 'ts': '2022-01-14 13:51:24'}

Consumed record from DEALER1_CARS with value {'user': 'William Diaz', 'rid': '3', 'manufacturer': 'Fiat', 'model': 'Tipo', 'plate': 'FGL1X62', 'status': 'START', 'ts': '2022-01-14 13:51:24'}

Consumed record from DEALER1_CARS with value {'user': 'Molly Davis', 'rid': '1', 'manufacturer': 'Fiat', 'model': 'Panda', 'plate': 'FJ7PUJJ', 'status': 'END', 'ts': '2022-01-14 13:51:24'}

Consumed record from DEALER1_CARS with value {'user': 'William Diaz', 'rid': '3', 'manufacturer': 'Fiat', 'model': 'Tipo', 'plate': 'FGL1X62', 'status': 'END', 'ts': '2022-01-14 13:51:24'}

Consum

In [ ]:
from confluent_kafka import Consumer, KafkaError
import json

c = Consumer(
    {
        'bootstrap.servers': 'kafka:9092',
        'group.id': 'test',
        'auto.offset.reset': 'earliest'
    }
)

c.subscribe(["DEALER2_VEHICLES","DEALER2_USERS"])

waiting = False

try:
    while True:
        msg = c.poll(1.0)                                        # retrieve records
        if msg is None:
            if waiting:
                print(".",end =" ")
            else:
                print("Waiting",end =" ")
                waiting = True
            continue
        elif msg.error():
            print('error: {}'.format(msg.error()))
            waiting = False
        else:
            value = json.loads(msg.value())                      # get the value of the message (json.loads)
            topic = msg.topic()                               # get the key of the message
            print("\nConsumed record from {} with value {}"
                  .format(topic, value))
            waiting = False
except KeyboardInterrupt:
    pass
finally:
    c.close()  


Consumed record from DEALER2_USERS with value {'userID': '3', 'name': 'Kevin Rodriguez'}

Consumed record from DEALER2_USERS with value {'userID': '4', 'name': 'Catherine Crandell'}

Consumed record from DEALER2_USERS with value {'userID': '1', 'name': 'Douglas Fitch'}

Consumed record from DEALER2_USERS with value {'userID': '2', 'name': 'William Diaz'}

Consumed record from DEALER2_USERS with value {'userID': '9', 'name': 'Mark Haws'}

Consumed record from DEALER2_USERS with value {'userID': '6', 'name': 'Wayne Flower'}

Consumed record from DEALER2_USERS with value {'userID': '7', 'name': 'Laura Baker'}

Consumed record from DEALER2_USERS with value {'userID': '8', 'name': 'Jane McPeak'}

Consumed record from DEALER2_USERS with value {'userID': '5', 'name': 'Richard Tillman'}

Consumed record from DEALER2_USERS with value {'userID': '10', 'name': 'Jeanie Morgan'}

Consumed record from DEALER2_VEHICLES with value {'userID': '3', 'type': 'car,', 'manifacturer': 'Audi', 'model': 'A3',

%6|1642019618.288|FAIL|rdkafka#consumer-3| [thrd:GroupCoordinator]: GroupCoordinator: kafka:9092: Disconnected (after 79341ms in state UP)
%4|1642019618.289|FAIL|rdkafka#consumer-3| [thrd:kafka:9092/bootstrap]: kafka:9092/1: Disconnected (after 79345ms in state UP)
%6|1642019618.290|FAIL|rdkafka#consumer-3| [thrd:kafka:9092/bootstrap]: kafka:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY)
%6|1642019618.290|FAIL|rdkafka#consumer-3| [thrd:GroupCoordinator]: GroupCoordinator: kafka:9092: Disconnected (after 1ms in state APIVERSION_QUERY, 1 identical error(s) suppressed)
%3|1642019618.808|FAIL|rdkafka#producer-1| [thrd:kafka:9092/bootstrap]: kafka:9092/1: Failed to resolve 'kafka:9092': Name or service not known (after 154ms in state CONNECT)


. . 

%3|1642019620.666|FAIL|rdkafka#producer-1| [thrd:kafka:9092/bootstrap]: kafka:9092/1: Failed to resolve 'kafka:9092': Name or service not known (after 12ms in state CONNECT, 1 identical error(s) suppressed)


. . . . 

...

. 

%3|1642019933.266|FAIL|rdkafka#consumer-3| [thrd:kafka:9092/bootstrap]: kafka:9092/1: Connect to ipv4#172.20.0.6:9092 failed: Connection refused (after 1034ms in state CONNECT)
%3|1642019933.299|FAIL|rdkafka#consumer-3| [thrd:GroupCoordinator]: GroupCoordinator: kafka:9092: Connect to ipv4#172.20.0.6:9092 failed: Connection refused (after 7062ms in state CONNECT, 1 identical error(s) suppressed)


. . . . . . . . . . . . 

%4|1642019945.306|CLUSTERID|rdkafka#producer-1| [thrd:main]: Broker kafka:9092/1 reports different ClusterId "aQbqAZPyRWOYMVeUZbNtKQ" than previously known "ssvyGL-lR56DLcPW_jZ-xQ": a client must not be simultaneously connected to multiple clusters
%4|1642019945.312|CLUSTERID|rdkafka#consumer-3| [thrd:main]: Broker kafka:9092/1 reports different ClusterId "aQbqAZPyRWOYMVeUZbNtKQ" than previously known "ssvyGL-lR56DLcPW_jZ-xQ": a client must not be simultaneously connected to multiple clusters


. . . . . . . . . . . . . . . 
Consumed record from DEALER2_USERS with value {'userID': '3', 'name': 'Kevin Rodriguez'}

Consumed record from DEALER2_USERS with value {'userID': '4', 'name': 'Catherine Crandell'}

Consumed record from DEALER2_USERS with value {'userID': '1', 'name': 'Douglas Fitch'}

Consumed record from DEALER2_USERS with value {'userID': '2', 'name': 'William Diaz'}

Consumed record from DEALER2_USERS with value {'userID': '9', 'name': 'Mark Haws'}

Consumed record from DEALER2_USERS with value {'userID': '6', 'name': 'Wayne Flower'}

Consumed record from DEALER2_USERS with value {'userID': '7', 'name': 'Laura Baker'}

Consumed record from DEALER2_USERS with value {'userID': '8', 'name': 'Jane McPeak'}

Consumed record from DEALER2_USERS with value {'userID': '5', 'name': 'Richard Tillman'}

Consumed record from DEALER2_USERS with value {'userID': '10', 'name': 'Jeanie Morgan'}

Consumed record from DEALER2_VEHICLES with value {'userID': '3', 'type': 'car,', 'manifac